In [55]:
import pandas as pd
import numpy as np


# Домашнее задание

Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA тест, проверьте данные на нормальность и на наличие корреляции. Для этого объедините 2 массива в DataFrame. Обоснуйте выбор теста на корреляцию. Сделайте вывод по гипотезе.
 
 
Также ответьте на вопрос:
Какова разница в среднем размере мидии в зависимости от города-производителя. Обоснуйте выбор теста. Сделайте вывод по гипотезе.


### 1) Объединяем массивы в DataFrame

In [56]:
#Из-за разницы в длине массивов, запишем пропуск как None
df = pd.DataFrame({
    'petersburg': [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105, None],
    'magadan': [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]
})

#Заполним None медианой
df = df.fillna(value={'petersburg': df['petersburg'].median()})

display(df)

,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,0.1016,0.0689


### 2) Для выбора теста на корреляцию сначала проверим данные на нормальность через Тест Шапиро-Уилка, который определяет, насколько вероятно, что данные были получены из распределения Гаусса

In [57]:
from scipy.stats import shapiro

In [58]:
alpha = 0.05
H0 = 'Данные распределены нормально'
H1 = 'Данные не распределены нормально'

_, p = shapiro(df)
print('p=%.3f' % p)
if p > alpha:
    print(H0)
else:
    print(H1)

p=0.101
Данные распределены нормально


### 3) Поскольку данные распределены нормально, для проверки нулевой гипотизы, мы должны выбрать параметрический тест.
### Для количественных данных с нормальным распределением, поскольку у нас 2 непрерывных переменных, мы выбираем корреляционный тест Пирсона.

In [59]:
from scipy.stats import pearsonr

In [75]:
H0 = 'Нет значимой корреляции между размерами мидий'
H1 = 'Есть значимая корреляция между размерами мидий'

def pearson_test(df):
    print('\n' + '*** Результат теста Пирсона ***')
    test_results = pearsonr(df['petersburg'], df['magadan'])
    p = round(test_results[1],2)
    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотизу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотизу. {H1}")
pearson_test(df)


*** Результат теста Пирсона ***
0.54 > 0.05. Мы не можем отвергнуть нулевую гипотизу. Нет значимой корреляции между размерами мидий


## 4) Теперь проведём тест для сравнения двух гупп.
## Данные распределены нормально, поэтому нужен параметрический тест.
## Зависимая переменная количественная, поэтому нужен тест на сравнение выборки.
## Поскольку сравнивается 2 группы с числом объектов менее 30, выбираем Т-тест 

In [61]:
# Независимый Т-тест
from scipy.stats import ttest_ind

In [76]:

H0 = 'Нет значимой разницы между средними размерами мидий'
H1 = 'Есть значимая разница между средними размерами мидий'

def t_test(df):
    print('\n' + '*** Результат независимого Т-теста ***')
    test_results = ttest_ind(df['petersburg'], df['magadan'], equal_var=True)
    p = round(test_results[1],2)
    if p>alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотизу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотизу. {H1}")
t_test(df)


*** Результат независимого Т-теста ***
0.0 <= 0.05. Мы отвергаем нулевую гипотизу. Есть значимая разница между средними размерами мидий


### 5) Какова разница в среднем размере мидии в зависимости от города-производителя?

In [63]:
mean_p = df['petersburg'].mean()
mean_m = df['magadan'].mean()
print(mean_p - mean_m)

0.0252


### Мы отвергаем нулевую гипотизу. Есть значимая разница между средними размерами мидий.
### Разница в среднем размере мидий, выращенных в Петербурге и Магадане составлят 0.025